# Exercise 5 - Automatically find Exchange links for DWM Query

In this demo we will see how to combine DWM and Iknaio to automatically find connections to exchanges given a set of crypto addresses mentioned in some genre of darkweb sites. Our topic today is CSAM.

## Preparations

First, we install the graphsense-python package and define an API-key. An API-key for the [GraphSense](https://graphsense.github.io/) instance hosted by [Iknaio](https://www.ikna.io/) can be requested by sending an email to [contact@iknaio.com](contact@iknaio.com).

In [3]:
!pip install graphsense-python seaborn tqdm json-api-doc openpyxl

import graphsense
from graphsense.api import bulk_api, general_api

import json
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime


# expect some local libs 
# files can be found at (https://github.com/iknaio/iknaio-api-tutorial/tree/main/standalone)
# in google colab they are downloaded automatically
try:
    from google.colab import userdata
    libs = ["dwm", "gs"]
    for l in libs:
        response = requests.get(f"https://raw.githubusercontent.com/iknaio/iknaio-api-tutorial/refs/heads/main/standalone/{l}.py")
        with open(f"{l}.py", "w") as f:
            f.write(response.text)
except ImportError:
    pass

import dwm
import gs

def ts_to_pds(ts):
    return datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
try:
    from google.colab import userdata
    
    # Expected structure of the file
    secrets = {
         "gs-api-key" : userdata.get('gs-api-key'),
         "dwm-credentials" : {"username": userdata.get('dwm-credentials-username'), "password": userdata.get('dwm-credentials-password')}
    }
except ImportError:
    with open("secrets.json") as f:
        secrets = json.load(f)

# We only work with BTC in this example
CURRENCY = 'btc'

# 1. DWM - Load addresses mentioned on Darkweb sites

In [5]:
# Request authentication token
headers = dwm.authenticate_api(secrets['dwm-credentials'])

## 1.a Load domains with a specific title

In [ ]:
# Collect domains related to title
title = "Alice with violence CP"

df_domains_all = dwm.get_domains_by_title(title, headers)
df_domains_all[["domain_url", "title", "status", "discovered_at"]].head(10)

KeyboardInterrupt: 

### Only Keep only offline domains

In [9]:
# only keep offline domains
# initial exploration has shown that flows from online websites all go trough service (tbdice)
# thus are hard to trace
df_domains = df_domains_all.query("status!='online'")
nr_domains = len(df_domains)
print(f"We have found {nr_domains} offline domains with title: {title}")

We have found 2383 domains with title: Alice with violence CP


## 1.b Get crypto addresses on the domains

In [ ]:
# Caution long-running (takes up to 1h)
df_cryptos_all = dwm.get_crypto_addresses_for_domains(df_domains, headers)

df_cryptos_all.head(5)

Processing domains: 100%|██████████| 2383/2383 [25:15<00:00,  1.57it/s]  


### Only keep BTC addresses

In [10]:
df_cryptos = df_cryptos_all.query(f"type=='{CURRENCY.upper()}'")
unique_addresses = len(df_cryptos["address"].unique())
print(f"We have found {len(df_cryptos)} BTC-addresses on these domains {unique_addresses} of which are unique")

We have found 37090 BTC-addresses on these domains 23626 of which are unique


### Save results

In [11]:
# save output in an excel file
with pd.ExcelWriter("alice_dwm.xlsx") as writer:
    df_domains.to_excel(writer, sheet_name="Domains", index=False)
    df_cryptos.to_excel(writer, sheet_name="Crypto-Assets", index=False)

# Save unique addresses in a CSV file
df_cryptos[["address"]].drop_duplicates(subset=["address"]).to_csv("addresses.csv")

# 2. Iknaio - Address Usage and Finding Exitpoints

In [7]:
configuration = graphsense.Configuration(
    host = "https://api.ikna.io/",
    api_key = {
        'api_key': secrets["gs-api-key"]
    }
)

We can test whether or client works, by checking what data the GraphSense endpoint provides

In [8]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = general_api.GeneralApi(api_client)
    api_response = api_instance.get_statistics()
    display({x['name']:x['no_blocks'] for x in api_response['currencies']})

{'btc': 879079,
 'bch': 880846,
 'ltc': 2826526,
 'zec': 2784842,
 'eth': 21615968,
 'trx': 68635588}

# 2.a How many of the found addresses are used?

Instead of querying each address individually, we just pass the dataframe of the known addresses.

In [10]:
seed_addresses = pd.read_csv("addresses.csv")

respAddrDF = gs.get_csv(configuration, "get_address", CURRENCY, {
                    'address': seed_addresses['address'].to_list()
                })

used_addresses = respAddrDF[["address", "balance_eur", "total_received_eur", "total_spent_eur", "in_degree", "out_degree", "no_incoming_txs", "no_outgoing_txs", "first_tx_timestamp", "last_tx_timestamp", "entity"]].dropna()
used_addresses.head(5)
used_addresses[["address"]].to_csv("used_addresses.csv")

In [11]:
print(f"{len(used_addresses)} addresses received {sum(used_addresses['total_received_eur']):.2f} EUR, Balance {sum(used_addresses['balance_eur']):.2f} EUR")
print(f"Activity period of the addresses was: {ts_to_pds(min(used_addresses['first_tx_timestamp']))} to {ts_to_pds(max(used_addresses['last_tx_timestamp']))}")

345 addresses received 32192.64 EUR, Balance 642.38 EUR
Activity period of the addresses was: 2019-12-13 13:14:24 to 2025-01-04 22:50:09


# 2.b Are there direct links to exchanges?

Direct link here means without any hops in-between directly form the source address found in the darkweb.

In [13]:
respAddrNbrDF = gs.get_csv(configuration, "list_address_neighbors", CURRENCY, {
                    'address': used_addresses['address'].to_list(),
                    'direction': 'out',
                    'include_labels': True
                })

with_label_direct = respAddrNbrDF.query("labels.notnull()")

with_outgoing_direct = respAddrNbrDF.query("_info != 'no data'")

print(f"We have found {len(with_outgoing_direct)} outgoing neighbors, {len(with_label_direct)} are known")

with_label_direct[["_request_address", "address_address", "labels"]]

We have found 621 outgoing neighbors, 46 are known


,_request_address,address_address,labels
19,3FWgtSB2rfMjtP7dVUxSfb2WTTk1APVwye,3FtduGyvPXsJV3Z44MgRThoensbNopWE6Z,Dark Web
25,3H5kaAbBLmpXf9yqdfiZefYrChR3XAnSA1,bc1qlduqs0yax4zftfuez8frvpenlxk9gmfpmaayyr,Wasabi
54,3PVmuqk5xDUaMv7hV8RyBqhf1xEf9DAp1o,3FtduGyvPXsJV3Z44MgRThoensbNopWE6Z,Dark Web
56,3F7SA4dEy4mUY4LQK5ieQtCAwxJM8MF4Y4,3FtduGyvPXsJV3Z44MgRThoensbNopWE6Z,Dark Web
58,3QDpGxLhiqJCWtrH174nQsmWTaKqifRRRw,3FtduGyvPXsJV3Z44MgRThoensbNopWE6Z,Dark Web
89,348nRKTtfam4RVPL97TBzf6pxGjjFVpghP,3FtduGyvPXsJV3Z44MgRThoensbNopWE6Z,Dark Web
120,3L7S3aDjdHexMoBmAx3Pap2JfocTE5efBc,bc1qlduqs0yax4zftfuez8frvpenlxk9gmfpmaayyr,Wasabi
130,14smqvFY4PgMpjmAHmfg4X824bh5to6HXn,1QJcNN3TdF5yQmbbXJtuBTtoeNABRJ2hbJ,Dark Web
138,32wH4LfP9nWzRheH6v64XtcUMxJUyBxTyz,3FtduGyvPXsJV3Z44MgRThoensbNopWE6Z,Dark Web
164,1NSy21LjaF3dPGWBVX9JfxxGTMCX3YoXjj,3CTuxSTzkTayenjZQjpzru3uXwHmRjS1yT,Dark Web


In [44]:
binance_direct = gs.get_csv(configuration, "list_address_links", CURRENCY, {
                    'address': ["bc1qwt7n66u465rfwf4gqwk20ywxlqwmrv8zkwf03s"],
                    'neighbor': ['bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq'],
                })
binance_direct

for i, r in binance_direct.iterrows():
    print(f"https://app.ikna.io/pathfinder/{r['currency']}/path/PA_{r['_request_address']},T_{r['tx_hash']},HA_{r['_request_neighbor']}")

https://app.ikna.io/pathfinder/btc/path/PA_bc1qwt7n66u465rfwf4gqwk20ywxlqwmrv8zkwf03s,T_d9344a24b30d9650bfaeb116f32203f732faa0205d1d42d0d0ab4dbd6be58611,HA_bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq


# 2.c Can I find links to exchange via Clusters?

We now fetch the cluster for each address

In [22]:
respEntityDF = gs.get_csv(configuration, "get_entity", CURRENCY, {
                                     'entity': used_addresses['entity'].drop_duplicates().to_list(),
                                     "exclude_best_address_tag": True
                                     })

clusters = respEntityDF[
    ["best_address_tag_label",
     "root_address",
     "no_addresses",
     "best_address_tag_label",
     "balance_eur",
     "total_received_eur",
     "total_spent_eur",
     "first_tx_timestamp",
     "last_tx_timestamp"]
     ]

print(f"{sum(clusters['no_addresses'])-len(used_addresses)} new addresses have been found. In {len(clusters)} clusters. They received {sum(clusters['total_received_eur']):.2f} EUR, Balance {sum(clusters['balance_eur']):.2f} EUR")
print(f"Activity period of the cluster addresses were: {ts_to_pds(min(clusters['first_tx_timestamp']))} to {ts_to_pds(max(clusters['last_tx_timestamp']))}")
clusters.query("best_address_tag_label.notnull()")

12726 new addresses have been found. In 91 clusters. They received 1378571.12 EUR, Balance 6607.20 EUR
Activity period of the cluster addresses were: 2014-08-09 23:48:57 to 2025-01-12 13:50:05


,best_address_tag_label,root_address,no_addresses,best_address_tag_label,balance_eur,total_received_eur,total_spent_eur,first_tx_timestamp,last_tx_timestamp


In [47]:
respEntityDF["no_addresses"].sum()

13071

In [ ]:
# get nbrs of the entities of the seed addresses
respEntityNbrs = gs.get_csv(configuration, "list_entity_neighbors", CURRENCY, {
                    'entity': used_addresses['entity'].drop_duplicates().to_list(),
                    'direction': 'out',
                    'include_labels': True
                })

respEntityNbrs["_request_entity"] = respEntityNbrs["_request_entity"].apply(int)
nbrs_id = respEntityNbrs["_request_entity"].unique().tolist()

# get detailed entities data with tags
respNbrsEntitiesDF = gs.get_csv(configuration, "get_entity", CURRENCY, {
                                     'entity': nbrs_id,
                                     "exclude_best_address_tag": True
                                     })

clustersNbrs = respEntityDF[
    ["best_address_tag_label",
     "root_address",
     "no_addresses",
     "best_address_tag_label",
     "balance_eur",
     "total_received_eur",
     "total_spent_eur",
     "first_tx_timestamp",
     "last_tx_timestamp"]
     ]


clustersNbrs.query("best_address_tag_label.notnull()")

,best_address_tag_label,root_address,no_addresses,best_address_tag_label,balance_eur,total_received_eur,total_spent_eur,first_tx_timestamp,last_tx_timestamp


# 2.d What if I look at multiple hops (using QL)? Are there any exchanges?

In [41]:
import shelve

addresses_used_list = used_addresses["address"].to_list()

with shelve.open('trace_cache.db') as cache:
    x = gs.get_QL_results_many(addresses_used_list, CURRENCY, {"Authorization": secrets["gs-api-key"]}, cache)

relevant_traces = sorted([y for y in x if y['nr_pathes_found'] > 0], key=lambda x: x["pct_traced_to_exchange"])

print(f"Found {len(relevant_traces)} paths to exchange")

gs.get_Pathfinder_link_from_ql_result(relevant_traces[0])

Searching centralized exchange connections for addresses:  30%|██▉       | 103/345 [04:06<09:57,  2.47s/it]

In [42]:
gs.get_Pathfinder_link_from_ql_result(relevant_traces[1])

'https://app.ikna.io/pathfinder/btc/path/PA_bc1q7zu3ffsnm55ltss0r0yj5vj6z5766ngdm34ast,T_1c81da30153892bec6cbe680634d83221030800d21d4e4a539eae31667363124,HA_bc1q8gqc2fmycy3mxw3pjjkwtawlspqwgvhgxrl5s0,T_0b68df2fc7bedc5cb1fd7cd27783b5b354ee8ac5e2bbcca087686786eb6874cf,HA_bc1q2scftr55wqxjggj34hu9z9hhz3emdtejy5a04q,T_9238d83ea4ff11f7f2adc86a75c0abd4ad8eb7cc0ebcc67459ea53a9e0b72ac3,HA_bc1qns9f7yfx3ry9lj6yz7c9er0vwa0ye2eklpzqfw'